In [ ]:
import sys
from PyQt5.QtWidgets import QMainWindow, QApplication
from PyQt5 import uic
import numpy as np
from scipy.stats import norm

qtCreatorFile = "bs.ui"
Ui_MainWindow, QtBaseClass = uic.loadUiType(qtCreatorFile)

class Main(QMainWindow, Ui_MainWindow):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.Calculate.clicked.connect(self.BS)
    
    def BS(self):
        S=float(self.StockPrice.text())
        K=float(self.ExercisePrice.text())
        T=float(self.Time.text())
        V=float(self.Volatility.text())
        R=float(self.InterestRate.text())
        Q=float(self.YieldRate.text())
        M=int(self.M1.text())
        N=int(self.N1.text())
        v=V/100 # V=sigma
        r=R/100
        q=Q/100
        d1=(np.log(S/K)+(r-q+v**2/2)*T)/(v*np.sqrt(T))
        d2=d1-(v*np.sqrt(T))
        Smax = 2*K
        dS = Smax/M
        dt = T/N
        C=round(S*norm.cdf(d1)*np.exp(-q*T)-norm.cdf(d2)*K*np.exp(-r*T),2)
        p=round(K*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp(-q*T)*norm.cdf(
                -d1),2)
        k=int(S/dS)
        
        b= np.zeros((M+1,M+1))
        for x in range(1,M):
            b[x][x]= (1)-dt*(v**2*x**2+r)
            b[x][x-1]= (1/2)*dt*(v**2*x**2-r*x)
            b[x][x+1]= (1/2)*dt*(v**2*x**2+r*x)
        b[0][0]=1
        b[M][M]=1
        np_b = np.array(b)
        
        c = np.zeros((M+1,M+1))
        for x in range(1,M):
            c[x][x]= (1)+dt*(v**2*x**2+r)
            c[x][x-1]= (1/2)*dt*((r-q)*x-v**2*x**2)
            c[x][x+1]= (-1/2)*dt*((r-q)*x+v**2*x**2)
        c[0][0]=1
        c[M][M]=1
        np_c=np.array(c)
        np_cinv= np.linalg.inv(np_c)
        
        d = np.zeros((M+1,M+1))
        for x in range(1,M):
            d[x][x]= 1+(-1/2)*dt*(v**2*x**2+r)
            d[x][x-1]= (1/4)*dt*(v**2*x**2-(r-q)*x)
            d[x][x+1]= (1/4)*dt*((r-q)*x+v**2*x**2)
            d[0][0]=1
            d[M][M]=1
        np_d=np.array(d)
        
        e = np.zeros((M+1,M+1))
        for x in range(1,M):
            e[x][x]= 1-(-1/2)*dt*(v**2*x**2+r)
            e[x][x-1]= (-1/4)*dt*(v**2*x**2-(r-q)*x)
            e[x][x+1]= (-1/4)*dt*((r-q)*x+v**2*x**2)
        e[0][0]=1
        e[M][M]=1
        np_e=np.array(e)
        np_einv= np.linalg.inv(np_e)
        
        table = np.zeros((M+1,N+1))
        tableC = np.array(table)
        tableP = np.array(table)
        fmax=np.array([i*dS for i in range(M+1)])
        tableC[:,N] = np.maximum(fmax-K,0)
        tableP[:,N] = np.maximum(K-fmax,0)
        
        np_tableEC=np.array(tableC)
        np_tableEP=np.array(tableP)
        
        np_tableIC=np.array(tableC)
        np_tableIP=np.array(tableP)
        
        np_tableCNC=np.array(tableC)
        np_tableCNP=np.array(tableP)
        
        for y in range(N-1,-1,-1):
            np_tableEC[:,y]=np_b.dot(np_tableEC[:,y+1])
            np_tableEP[:,y]=np_b.dot(np_tableEP[:,y+1])
            np_tableIC[:,y]=(np_cinv).dot(np_tableIC[:,y+1])
            np_tableIP[:,y]=(np_cinv).dot(np_tableIP[:,y+1])
            vbc=(np_d).dot(np_tableCNC[:,y+1])
            np_tableCNC[:,y]=np_einv.dot(vbc)
            vbp=(np_d).dot(np_tableCNP[:,y+1])
            np_tableCNP[:,y]=np_einv.dot(vbp)
            
        ECO =round(np_tableEC[k,0]+(np_tableEC[k+1,0]-
                    np_tableEC[k,0])/dS*(S-k*dS),2)
        EPO =round(np_tableEP[k,0]+(np_tableEP[k+1,0]-
                    np_tableEP[k,0])/dS*(S-k*dS),2)
        ICO =round(np_tableIC[k,0]+(np_tableIC[k+1,0]-
                    np_tableIC[k,0])/dS*(S-k*dS),2)
        IPO =round(np_tableIP[k,0]+(np_tableIP[k+1,0]-
                    np_tableIP[k,0])/dS*(S-k*dS),2)
        CNCO = round( np_tableCNC[k,0]+(np_tableCNC[k+1,0]-
                    np_tableCNC[k,0])/dS*(S-k*dS),2)
        CNPO = round( np_tableCNP[k,0]+(np_tableCNP[k+1,0]-
                    np_tableCNP[k,0])/dS*(S-k*dS),2)
        print('Black-Scholes option price, Call:',C,'Put:',p)
        print('Explicit method option price, Call:',ECO,'Put:',EPO)
        print('Implicit method option price, Call:',ICO,'Put:',IPO)
        print('Crank-Nicolson method option price,Call:',CNCO,'Put:',CNPO)
        
        self.no_C.setText(str(C))
        self.no_P.setText(str(p))
        self.E_C.setText(str(ECO))
        self.E_P.setText(str(EPO))
        self.I_C.setText(str(ICO))
        self.I_P.setText(str(IPO))
        self.CN_C.setText(str(CNCO))
        self.CN_P.setText(str(CNPO))
        
        if C>ECO:
            self.E_C.setStyleSheet("color: red;")
        else:
            self.E_C.setStyleSheet("color: green;")
        if C>ICO:
            self.I_C.setStyleSheet("color: red;")
        else:
            self.I_C.setStyleSheet("color: green;")
        if C>CNCO:
            self.CN_C.setStyleSheet("color: red;")
        else:
            self.CN_C.setStyleSheet("color: green;")
        if p>EPO:
            self.E_P.setStyleSheet("color: red;")
        else:
            self.E_P.setStyleSheet("color: green;")
        if p>IPO:
            self.I_P.setStyleSheet("color: red;")
        else:
            self.I_P.setStyleSheet("color: green;")
        if p>CNPO:
            self.CN_P.setStyleSheet("color: red;")
        else:
            self.CN_P.setStyleSheet("color: green;")

if __name__=='__main__':
    app=QApplication(sys.argv)
    main=Main()
    main.show()
    sys.exit(app.exec_())